In [ ]:
import time
import random
from colorsys import hsv_to_rgb
import board
from digitalio import DigitalInOut, Direction
from PIL import Image, ImageDraw, ImageFont
from adafruit_rgb_display import st7789
import numpy as np
import os

class Joystick:
    def __init__(self):
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000

        self.spi = board.SPI()
        self.disp = st7789.ST7789(
                    self.spi,
                    height=240,
                    y_offset=80,
                    rotation=180,
                    cs=self.cs_pin,
                    dc=self.dc_pin,
                    rst=self.reset_pin,
                    baudrate=self.BAUDRATE,
                    )

        # Input pins
        self.button_A = DigitalInOut(board.D5)
        self.button_A.direction = Direction.INPUT

        self.button_B = DigitalInOut(board.D6)
        self.button_B.direction = Direction.INPUT

        self.button_L = DigitalInOut(board.D27)
        self.button_L.direction = Direction.INPUT

        self.button_R = DigitalInOut(board.D23)
        self.button_R.direction = Direction.INPUT

        self.button_U = DigitalInOut(board.D17)
        self.button_U.direction = Direction.INPUT

        self.button_D = DigitalInOut(board.D22)
        self.button_D.direction = Direction.INPUT

        self.button_C = DigitalInOut(board.D4)
        self.button_C.direction = Direction.INPUT

        # Turn on the Backlight
        self.backlight = DigitalInOut(board.D26)
        self.backlight.switch_to_output()
        self.backlight.value = True

        # Create blank image for drawing
        self.width = self.disp.width
        self.height = self.disp.height

    def show_opening_animation(self):
        opening_frames = []
        for i in range(1, 4):  # opening1.jpeg, opening2.jpeg, opening3.jpeg
            frame_path = f"/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/opening/opening{i}.jpeg"  # 경로 수정
            try:
                frame_image = Image.open(frame_path)
                opening_frames.append(frame_image)
            except FileNotFoundError:
                print(f"Error: {frame_path} not found.")
                return  # 파일이 없으면 메서드 종료

        for frame in opening_frames:
            resized_frame = frame.resize((self.disp.width, self.disp.height))
            self.disp.image(resized_frame)
            time.sleep(0.5)
        time.sleep(1)  # 오프닝 후 대기

class Character:
    def __init__(self, width, height):
        self.appearance = 'image'
        self.state = None
        self.width = width
        self.height = height
        
        # 기본 속성 설정
        self.position = np.array([width / 2 - 15, height / 2 - 15, width / 2 + 15, height / 2 + 15])
        self.outline = "#FFFFFF"
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        
        # 디폴트 이미지
        self.image_paths = {
            "right": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character-right.png",
            "left": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character-left.png",
            "up": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character_up.png",
            "down": "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/character/character-right.png",
        }

        # 디폴트 방향으로 이미지를 로드합니다.
        self.direction = "right"
        self.image = self.load_image(self.image_paths[self.direction])

        self.bullets = []  # 총알 목록 초기화

    def load_image(self, path):
        if os.path.isfile(path):
            return Image.open(path).convert("RGBA").resize((30, 30))  # 이미지 로드 및 크기 조정
        else:
            print(f"Error: Character image not found at {path}")
            return None  # 이미지가 없을 경우 None으로 설정

    def move(self, command=None):
        if command and command['move']:
            # 이동 전 위치와 경계를 체크
            if command['up_pressed']:
                if self.position[1] > 0:  # 위쪽 경계 체크
                    self.position[1] -= 5
                    self.position[3] -= 5
                    self.direction = "up"
            elif command['down_pressed']:
                if self.position[3] < self.height:  # 아래쪽 경계 체크
                    self.position[1] += 5
                    self.position[3] += 5
                    self.direction = "down"
            elif command['left_pressed']:
                if self.position[0] > 0:  # 왼쪽 경계 체크
                    self.position[0] -= 5
                    self.position[2] -= 5
                    self.direction = "left"
            elif command['right_pressed']:
                if self.position[2] < self.width:  # 오른쪽 경계 체크
                    self.position[0] += 5
                    self.position[2] += 5
                    self.direction = "right"

            # 이미지 로드
            self.image = self.load_image(self.image_paths[self.direction])

        # Center 업데이트
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
    
    def shoot(self, direction):
        bullet_position = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        bullet = Bullet(bullet_position, direction)
        self.bullets.append(bullet)  # 총알을 목록에 추가
    
class Enemy:
    def __init__(self, spawn_position):
        self.appearance = 'circle'
        self.state = 'alive'
        self.position = np.array([spawn_position[0] - 25, spawn_position[1] - 25, spawn_position[0] + 25, spawn_position[1] + 25])
        self.center = np.array([(self.position[0] + self.position[2]) / 2, (self.position[1] + self.position[3]) / 2])
        self.outline = "#00FF00"

class Bullet:
    def __init__(self, position, direction):
        self.appearance = 'rectangle'
        self.speed = 10
        self.damage = 10
        self.position = np.array([position[0] - 3, position[1] - 3, position[0] + 3, position[1] + 3])
        self.direction = direction
        self.state = None
        self.outline = "#0000FF"

    def move(self):
        if self.direction == "up":
            self.position[1] -= self.speed
            self.position[3] -= self.speed
        elif self.direction == "down":
            self.position[1] += self.speed
            self.position[3] += self.speed
        elif self.direction == "left":
            self.position[0] -= self.speed
            self.position[2] -= self.speed
        elif self.direction == "right":
            self.position[0] += self.speed
            self.position[2] += self.speed

    def collision_check(self, enemies):
        for enemy in enemies:
            if self.overlap(self.position, enemy.position):
                enemy.state = 'die'
                self.state = 'hit'

    def overlap(self, ego_position, other_position):
        return (ego_position[0] < other_position[2] and
                ego_position[2] > other_position[0] and
                ego_position[1] < other_position[3] and
                ego_position[3] > other_position[1])

class Game:
    def __init__(self):
        self.joystick = Joystick()
        self.current_stage = 1  # 기본 스테이지를 1로 설정
        self.background_image = self.load_background_image()
        self.character = Character(self.joystick.width, self.joystick.height)  # 캐릭터 초기화 (하나의 인스턴스로)

    def load_background_image(self):
        # 현재 스테이지에 따라 배경 이미지 경로 설정
        if self.current_stage == 1:
            image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage1.jpeg"
        elif self.current_stage == 2:
            image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage2.jpeg"
        elif self.current_stage == 3:
            image_path = "/home/rjhpi/Desktop/TA-ESW/PuzzleHunt/img/stage/stage3.jpeg"
        else:
            image_path = None

        if image_path is not None and os.path.isfile(image_path):
            return Image.open(image_path).resize((self.joystick.width, self.joystick.height))
        else:
            print(f"Error: {image_path} not found.")
            return None  # 이미지 로드 실패 시 None 반환

    def show_opening_animation(self):
        # Joystick의 show_opening_animation() 호출
        self.joystick.show_opening_animation()
        # 스테이지 변환 코드 추가
        self.background_image = self.load_background_image()  # 스테이지에 맞는 배경 이미지 로드

    def main_loop(self):
        while True:
            my_image = Image.new("RGB", (self.joystick.width, self.joystick.height))  # 도화지 생성
            my_draw = ImageDraw.Draw(my_image)
    
            command = {'move': False, 'up_pressed': False, 'down_pressed': False, 'left_pressed': False, 'right_pressed': False}
    
            # 조이스틱 입력 처리
            if not self.joystick.button_U.value:  # up pressed
                command['up_pressed'] = True
                command['move'] = True
            if not self.joystick.button_D.value:  # down pressed
                command['down_pressed'] = True
                command['move'] = True
            if not self.joystick.button_L.value:  # left pressed
                command['left_pressed'] = True
                command['move'] = True
            if not self.joystick.button_R.value:  # right pressed
                command['right_pressed'] = True
                command['move'] = True
    
            # 총알 발사 처리
            if not self.joystick.button_A.value:  # A 버튼이 눌리면 총알 발사
                direction = self.character.direction  # 현재 방향에 총알 발사
                self.character.shoot(direction)
    
            self.character.move(command)  # 캐릭터 이동 처리
    
            # 배경 이미지를 그립니다
            if self.background_image is not None:
                my_image.paste(self.background_image, (0, 0))  # 배경 이미지 붙여 넣기
            else:
                # 배경 이미지가 없을 경우 흰색으로 초기화
                my_draw.rectangle((0, 0, self.joystick.width, self.joystick.height), fill=(255, 255, 255))
    
            # 캐릭터 이미지를 그리기
            character_draw_position = (int(self.character.position[0]), int(self.character.position[1]))  # 캐릭터 위치
            if self.character.image is not None:
                my_image.paste(self.character.image, character_draw_position, self.character.image)  # 이미지 붙여넣기
            else:
                # 이미지가 없을 경우 흰색 원으로 대체
                my_draw.ellipse(tuple(self.character.position), outline=self.character.outline, fill=(0, 0, 0))  # 기본 원 그리기
    
            # 총알 이동 및 그리기
            for bullet in self.character.bullets:
                bullet.move()  # 총알 이동 처리
                bullet_position = (bullet.position[0], bullet.position[1], bullet.position[2], bullet.position[3])
                if bullet.state != 'hit':
                    my_draw.rectangle(bullet_position, outline=bullet.outline, fill=(0, 0, 255))  # 총알 그리기
    
            # 화면에 현재 이미지 디스플레이
            self.joystick.disp.image(my_image)


def main():
    game = Game()  # Game 객체 생성
    game.show_opening_animation()  # 오프닝 애니메이션 표시 및 스테이지 로드
    game.main_loop()  # 메인 루프 시작

if __name__ == "__main__":
    main()  # 메인 함수 실행